# Loading Control and Test Group Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

In [2]:
# Upload df_sorted (It includes all the test and control group and not null values in variation)
file_path = r'C:\Users\ALEV PC\Desktop\Project\Notebooks\df_sorted.csv'
df = pd.read_csv(file_path)
display(df)


,client_id,visitor_id,visit_id,process_step,date_time,Variation,end_of_the_step,time_diff
0,555,402506806_56087378777,637149525_38041617439_716659,start,2017-04-15 12:57:56,Test,2017-04-15 12:58:03,0 days 00:00:07
1,555,402506806_56087378777,637149525_38041617439_716659,step_1,2017-04-15 12:58:03,Test,2017-04-15 12:58:35,0 days 00:00:32
2,555,402506806_56087378777,637149525_38041617439_716659,step_2,2017-04-15 12:58:35,Test,2017-04-15 13:00:14,0 days 00:01:39
3,555,402506806_56087378777,637149525_38041617439_716659,step_3,2017-04-15 13:00:14,Test,2017-04-15 13:00:34,0 days 00:00:20
4,555,402506806_56087378777,637149525_38041617439_716659,confirm,2017-04-15 13:00:34,Test,NaN,NaN
...,...,...,...,...,...,...,...,...
321304,9999729,843385170_36953471821,493310979_9209676464_421146,step_1,2017-04-20 14:22:49,Test,2017-04-20 14:27:36,0 days 00:04:47
321305,9999729,843385170_36953471821,493310979_9209676464_421146,step_2,2017-04-20 14:27:36,Test,2017-04-20 14:28:57,0 days 00:01:21
321306,9999729,843385170_36953471821,493310979_9209676464_421146,start,2017-04-20 14:28:57,Test,NaN,NaN
321307,9999832,145538019_54444341400,472154369_16714624241_585315,start,2017-05-16 16:46:03,Test,2017-05-16 16:46:11,0 days 00:00:08


In [3]:
import pandas as pd

# Convert 'date_time' to datetime if it's not already
df['date_time'] = pd.to_datetime(df['date_time'])

# Group by 'client_id' and 'Variation'
grouped = df.groupby(['client_id', 'Variation'])

# Initialize an empty list to store the DataFrames
last_confirm_steps = []

# Iterate over each group
for group_key, group_df in grouped:
    # Filter for rows with 'confirm' as process_step
    confirm_steps = group_df[group_df['process_step'] == 'confirm']
    
    # Check if there are any 'confirm' steps
    if not confirm_steps.empty:
        # Get the last 'confirm' step based on the maximum 'date_time'
        last_confirm_step = confirm_steps.loc[confirm_steps['date_time'].idxmax()]
        
        # Append the last 'confirm' step DataFrame to the list
        last_confirm_steps.append(last_confirm_step)

# Check if there are any 'confirm' steps found
if last_confirm_steps:
    # Concatenate the DataFrames in the list into a single DataFrame
    result = pd.concat(last_confirm_steps, axis=1).T

    # Display the last 'confirm' step for each variation
    print(result)
else:
    print("No 'confirm' steps found.")



       client_id             visitor_id                      visit_id  \
4            555  402506806_56087378777  637149525_38041617439_716659   
9            647   66758770_53988066587   40369564_40101682850_311847   
33          1195  766842522_69992551638  393817425_39015278493_996341   
40          1197  753759429_54481946928   71862471_21202285428_848395   
47          1336  920624746_32603333901  614001770_19101025926_112779   
...          ...                    ...                           ...   
321274   9998851  660529049_81285947655  310744937_17135608286_213964   
321280   9998964  502193753_63097042630  699981922_52880598802_858635   
321285   9999009  212849256_28644888524  149965839_24823291142_245569   
321293   9999400  915967319_19082377501  288904166_90826265353_276104   
321302   9999729  834634258_21862004160  870243567_56915814033_814203   

       process_step            date_time Variation end_of_the_step time_diff  
4           confirm  2017-04-15 13:00:34    

In [ ]:
# Convert the list of last confirm steps to a DataFrame
last_confirm_steps_df = pd.DataFrame(last_confirm_steps)

# Count the total number of unique users for each variation
total_users_test = last_confirm_steps_df[last_confirm_steps_df['Variation'] == 'Test']['client_id'].nunique()
total_users_control = last_confirm_steps_df[last_confirm_steps_df['Variation'] == 'Control']['client_id'].nunique()

# Count the number of unique users who reached the "last_confirmed_step" for each variation
confirm_users_test = last_confirm_steps_df[(last_confirm_steps_df['Variation'] == 'Test') & (last_confirm_steps_df['process_step'] == 'confirm')]['client_id'].nunique()
confirm_users_control = last_confirm_steps_df[(last_confirm_steps_df['Variation'] == 'Control') & (last_confirm_steps_df['process_step'] == 'confirm')]['client_id'].nunique()

# Calculate the percentage of users who reached the "last_confirmed_step" for each variation
percentage_confirm_test = (confirm_users_test / total_users_test) * 100
percentage_confirm_control = (confirm_users_control / total_users_control) * 100

print("Percentage of users reaching the 'last_confirmed_step' for Test variation:", percentage_confirm_test)
print("Percentage of users reaching the 'last_confirmed_step' for Control variation:", percentage_confirm_control)


In [13]:
# Assuming the DataFrame is named df

# Filter for 'confirm' steps
confirm_steps = df[df['process_step'] == 'confirm']

# Group by 'Variation' and count the number of 'confirm' steps for each variation
confirm_counts = confirm_steps['Variation'].value_counts()

# Calculate the total number of 'confirm' steps for all variations
total_confirm = confirm_counts.sum()

# Calculate the percentage of 'confirm' steps for each variation
confirm_percentage = (confirm_counts / total_confirm) * 100

# Display the percentage of 'confirm' steps for each variation
print("Percentage of 'confirm' steps for each variation:")
confirm_percentage 


Percentage of 'confirm' steps for each variation:


Variation
Test       59.504476
Control    40.495524
Name: count, dtype: float64

In [14]:
import pandas as pd

# Assuming the DataFrame is named df

# Filter for 'confirm' steps
confirm_steps = df[df['process_step'] == 'confirm']

# Group by 'Variation' and count the number of 'confirm' steps for each variation
confirm_counts = confirm_steps['Variation'].value_counts()

# Calculate the total number of 'confirm' steps for all variations
total_confirm = confirm_counts.sum()

# Calculate the percentage of 'confirm' steps for each variation
confirm_percentage = (confirm_counts / total_confirm) * 100

# Display the percentage of 'confirm' steps for each variation
print("Percentage of 'confirm' steps for each variation:")
print(confirm_percentage)

# Define file path to save the confirmed percentage results
file_path = "confirmed_percentage.csv"

# Write the results to a CSV file
confirm_percentage.to_csv(file_path, header=True)

print(f"Confirmed percentage has been saved to '{file_path}' file.")


Percentage of 'confirm' steps for each variation:
Variation
Test       59.504476
Control    40.495524
Name: count, dtype: float64
Confirmed percentage has been saved to 'confirmed_percentage.csv' file.


In [17]:
# Two-sample t-test


import scipy.stats as stats

# Given data
percentage_confirm_test = 59.504476
percentage_confirm_control = 40.495524

# Perform two-sample t-test
t_statistic, p_value = stats.ttest_ind_from_stats(mean1=percentage_confirm_test, std1=0, nobs1=1, mean2=percentage_confirm_control, std2=0, nobs2=1)

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("There is a significant difference in the mean percentage of 'confirm' steps between the Test and Control variations.")
else:
    print("There is no significant difference in the mean percentage of 'confirm' steps between the Test and Control variations.")


There is no significant difference in the mean percentage of 'confirm' steps between the Test and Control variations.


C:\Users\ALEV PC\Desktop\Project\venv\Lib\site-packages\scipy\stats\_stats_py.py:6988: RuntimeWarning: invalid value encountered in divide
  svar = ((n1 - 1) * v1 + (n2 - 1) * v2) / df


In [18]:



import scipy.stats as stats

# Given data
percentage_confirm_test = [59.504476]  # List of percentage confirm values for the Test variation
percentage_confirm_control = [40.495524]  # List of percentage confirm values for the Control variation

# Perform Mann-Whitney U test
statistic, p_value = stats.mannwhitneyu(percentage_confirm_test, percentage_confirm_control)

# Interpret results
alpha = 0.05
if p_value < alpha:
    print("There is a significant difference in the mean percentage of 'confirm' steps between the Test and Control variations (Mann-Whitney U test).")
else:
    print("There is no significant difference in the mean percentage of 'confirm' steps between the Test and Control variations (Mann-Whitney U test).")


There is no significant difference in the mean percentage of 'confirm' steps between the Test and Control variations (Mann-Whitney U test).
